In [1]:
import pandas as pd
from functools import reduce

**Creating categories from worksheets located in the original file:**

In [2]:
### Dataframes containing filters: cases to count.
base = pd.read_excel('centros-investigadores.xlsx',sheetname=0)
infraestructura = pd.read_excel('centros-investigadores.xlsx',sheetname=1)
servicios = pd.read_excel('centros-investigadores.xlsx',sheetname=2)
formacion = pd.read_excel('centros-investigadores.xlsx',sheetname=3)
proyectos = pd.read_excel('centros-investigadores.xlsx',sheetname=4)
publicaciones = pd.read_excel('centros-investigadores.xlsx',sheetname=5)
vinculacion = pd.read_excel('centros-investigadores.xlsx',sheetname=6)
investigadores = pd.read_excel('centros-investigadores.xlsx',sheetname=7)

In [3]:
### Function to count frequencies of variables and then addit to the original dataframe ('base')
def countVar(dataframe, variable):
    def size(x):
        if(len(x) > 0): 
            return 1
    
    dataframe[variable] = infraestructura.apply(lambda row: size(row), axis=1)
    x = dataframe.groupby(['id_gi'])[variable].sum()
    x = x.reset_index()
    return x

In [17]:
### Function to count frequencies of national networks
def countNetNacional(dataframe, variable):
    def size(x):
        if(len(x) > 0): 
            return 1
    
    nd = dataframe.loc[dataframe['tipo_vinculo'] == 'nacional']
    
    nd[variable] = nd.apply(lambda row: size(row), axis=1)
    x = nd.groupby(['id_gi'])[variable].sum()
    x = x.reset_index()
    return x

### Function to count frequencies of international networks
def countNetInternacional(dataframe, variable):
    def size(x):
        if(len(x) > 0): 
            return 1
    
    nd = dataframe.loc[dataframe['tipo_vinculo'] == 'extranjera']
    
    nd[variable] = nd.apply(lambda row: size(row), axis=1)
    x = nd.groupby(['id_gi'])[variable].sum()
    x = x.reset_index()
    return x

In [19]:
### Creating new dataframes with variables included
labs = countVar(infraestructura,'labs')
services = countVar(servicios, 'services')
programmes = countVar(formacion, 'programmes')
projects = countVar(proyectos, 'projects')
publications = countVar(publicaciones, 'publications')
netnacional = countNetNacional(vinculacion, 'netnacional')
netextranera = countNetInternacional(vinculacion, 'netinternacional')
networks = countVar(vinculacion, 'networks')
researchers = countVar(investigadores, 'researchers')

### Merging all variables
frames = [labs,services,programmes,projects,publications,researchers,netnacional,netextranera,networks]
df_variables = reduce(lambda left,right: pd.merge(left, right, on=['id_gi'], how='outer'), frames)

### Mergin variables to Base dataframe
df_final = pd.merge(base, df_variables, how='inner', on='id_gi')
df_final.columns

### Exporting to CSV
df_final.to_csv('gi_filtros.csv', sep=',', index=False)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
! ls

Data-Handling.ipynb         gi_filtros.csv
centros-investigadores.xlsx gi_filtros.numbers
filtros.csv                 grupos.csv
filtros.tsv                 plantas_piloto.xlsx
flare.json                  template.key


### Process words inside 'lineas de investigacion'

In [6]:
import unidecode
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('spanish'))

In [7]:
stop.update(('.',',','(',')',':','-','/',';'))

In [8]:
df_lineas = pd.read_csv('gi_filtros.csv', sep=',')
df_lineas.columns

Index(['c_entidad', 'region', 'entidad', 'direccion_entidad',
       'telefono_entidad', 'mail_entidad', 'web_entidad', 'id_gi', 'grupo',
       'inocuidad', 'ingredientes', 'funcionalidad', 'linea_investigacion_gi',
       'inv_responsable', 'pregrado', 'postgrado', 'cargo',
       'telefono_inv_responsable', 'mail_inv_responsable', 'labs', 'services',
       'programmes', 'projects', 'publications', 'networks', 'researchers'],
      dtype='object')

In [9]:
### Create a list of words, removing stop words in spanish for lines of text.
def words(row):
    words = word_tokenize(row)
    lower = []
    for i in words:
        lower.append(unidecode.unidecode(i.lower()))
    return [i for i in lower if i not in stop]

df_lineas['linea_investigacion_gi'] = df_lineas['linea_investigacion_gi'].apply(words)

### Exporting to CSV
df_lineas.to_csv('filtros.csv', sep=' ', index=False)

### Produce 'flare' data format from dataframe for data visualizations.

In [10]:
import json

In [11]:
# CSV 2 flare.json
# convert a csv file to flare.json for use with many D3.js viz's
# This script creates outputs a flare.json file with 2 levels of nesting.
# For additional nested layers, add them in lines 32 - 47
# sample: http://bl.ocks.org/mbostock/1283663
# author: Andrew Heekin
# MIT License

# start a new flare.json document
# d = dict()
# d = {"name":"flare", "children": []}

# for line in df1.values:
#     the_parent = line[0]
#     the_child = line[1]
#     child_size = line[2]

#     # make a list of keys
#     keys_list = []
#     for item in d['children']:
#         keys_list.append(item['name'])

#     # if 'the_parent' is NOT a key in the flare.json yet, append it
#     if not the_parent in keys_list:
#         d['children'].append({"name":the_parent, "children":[{"name":the_child, "size":child_size}]})

#     # if 'the_parent' IS a key in the flare.json, add a new child to it
#     else:
#         d['children'][keys_list.index(the_parent)]['children'].append({"name":the_child, "size":child_size})


# # export the final result to a json file
# with open('flare.json', 'w') as outfile:
#     json.dump(d, outfile)